# Instrucciones

* Escoge 15 acciones del S&P 500 que esten listadas desde el 2015
* Crea 3 modelos, CAPM, 3 factores y 5 factores
* Obten coeficientes y R2
* Explica los resultados 

In [1]:
import pandas_datareader.data as web
import datetime
from sklearn.metrics import r2_score
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from IPython.display import display

# Definir las fechas de inicio y fin
start_date = datetime.datetime(2015, 1, 1)
end_date = datetime.datetime(2023, 12, 31)

# Descargar los factores Fama-French desde la web de Ken French
ff_factores = web.DataReader("F-F_Research_Data_5_Factors_2x3", "famafrench", start_date, end_date)

In [2]:
factors_df = ff_factores[0] / 100
factors_df.index = factors_df.index.to_timestamp()
factors_df.tail()

,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
2023-08-01,-0.0239,-0.0365,-0.0106,0.0343,-0.0237,0.0045
2023-09-01,-0.0524,-0.0180,0.0152,0.0186,-0.0083,0.0043
2023-10-01,-0.0319,-0.0405,0.0018,0.0247,-0.0065,0.0047
2023-11-01,0.0884,-0.0012,0.0164,-0.0391,-0.0100,0.0044
2023-12-01,0.0487,0.0732,0.0493,-0.0307,0.0132,0.0043


In [3]:
tickers = ['AAPL','MSFT','AMZN','WMT','COST',
          'JNJ','PFE','XOM','CVS','IBM',
          'V','AMD','MCD','KO','HP']
stock_data = yf.download(tickers, start_date, end_date, interval='1mo')['Adj Close']
rt = stock_data.pct_change().dropna()

[*********************100%%**********************]  15 of 15 completed


In [4]:
fama_french_df = pd.merge(factors_df, rt, on='Date')

In [5]:
# Correr todos los modelos
def run_all_models(data, ticker):
    flag_data = data.copy()
    flag_data[ticker + '_minus_RF'] = data[ticker] - data['RF']
    y = flag_data[[ticker + '_minus_RF']] 

    # CAPM 
    X_capm = flag_data[['Mkt-RF']]
    model_capm = LinearRegression().fit(X_capm, y)
    prediction_capm = model_capm.predict(X_capm)
    r2_capm = np.round(r2_score(y_pred=prediction_capm, y_true=y),4)
    summary_capm = {str(X_capm.keys().values): list(model_capm.coef_[0])}

    # 3 factor model 
    X_3factor = flag_data[['Mkt-RF', 'SMB', 'HML']]
    model_3factor = LinearRegression().fit(X_3factor, y)
    prediction_3factor = model_3factor.predict(X_3factor)
    r2_3factor = np.round(r2_score(y_pred=prediction_3factor, y_true=y),4)
    summary_3factor = {str(X_3factor.keys().values): list(model_3factor.coef_[0])}
    
    
    # 5 factor
    X_5factor = flag_data[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]
    model_5factor = LinearRegression().fit(X_5factor, y)
    prediction_5factor = model_5factor.predict(X_5factor)
    r2_5factor = np.round(r2_score(y_pred=prediction_5factor, y_true=y),4)
    summary_5factor = {str(X_5factor.keys().values): list(model_5factor.coef_[0])}
    
    summary = {
        'CAPM': {'BETAS': summary_capm, 'R2': r2_capm},
        '3 factor': {'BETAS': summary_3factor, 'R2': r2_3factor},
        '5 factor': {'BETAS': summary_5factor, 'R2': r2_5factor}

    }
    return summary

In [6]:
for i in range(len(tickers)):
    print(tickers[i])
    x = run_all_models(fama_french_df, tickers[i])
    display(x)
    print()

AAPL


{'CAPM': {'BETAS': {"['Mkt-RF']": [1.2139143124805247]}, 'R2': 0.4899},
 '3 factor': {'BETAS': {"['Mkt-RF' 'SMB' 'HML']": [1.2957946599992973,
    -0.355418360011865,
    -0.4705908785040141]},
  'R2': 0.5705},
 '5 factor': {'BETAS': {"['Mkt-RF' 'SMB' 'HML' 'RMW' 'CMA']": [1.2592218505101374,
    -0.19593280177661854,
    -0.53771932421899,
    0.38387386099178783,
    0.0009714315738426627]},
  'R2': 0.5781}}


MSFT


{'CAPM': {'BETAS': {"['Mkt-RF']": [0.8916677588124616]}, 'R2': 0.4536},
 '3 factor': {'BETAS': {"['Mkt-RF' 'SMB' 'HML']": [1.0508444704513755,
    -0.7590247154409094,
    -0.2897045528358261]},
  'R2': 0.6363},
 '5 factor': {'BETAS': {"['Mkt-RF' 'SMB' 'HML' 'RMW' 'CMA']": [1.0349693562466606,
    -0.8387601441585714,
    -0.14504009045941216,
    -0.11833908787073731,
    -0.2782759876412632]},
  'R2': 0.644}}


AMZN


...

{'CAPM': {'BETAS': {"['Mkt-RF']": [0.6084365144073168]}, 'R2': 0.3263},
 '3 factor': {'BETAS': {"['Mkt-RF' 'SMB' 'HML']": [0.7198975031896178,
    -0.5838921375284354,
    0.2782242652181516]},
  'R2': 0.4298},
 '5 factor': {'BETAS': {"['Mkt-RF' 'SMB' 'HML' 'RMW' 'CMA']": [0.7156876831587833,
    -0.42839898896608264,
    0.11049478731019896,
    0.3065266182287738,
    0.2566802119229208]},
  'R2': 0.4515}}


KO


{'CAPM': {'BETAS': {"['Mkt-RF']": [0.5042959044121006]}, 'R2': 0.2497},
 '3 factor': {'BETAS': {"['Mkt-RF' 'SMB' 'HML']": [0.648975378748007,
    -0.763150481419088,
    0.4092742421526173]},
  'R2': 0.4593},
 '5 factor': {'BETAS': {"['Mkt-RF' 'SMB' 'HML' 'RMW' 'CMA']": [0.6425460834753397,
    -0.6197340760576654,
    0.26285405618909297,
    0.2882045272486174,
    0.21603668870074874]},
  'R2': 0.479}}


HP


{'CAPM': {'BETAS': {"['Mkt-RF']": [1.5548799257163037]}, 'R2': 0.2749},
 '3 factor': {'BETAS': {"['Mkt-RF' 'SMB' 'HML']": [1.2914687236356945,
    1.1410706246257174,
    1.5351992758761281]},
  'R2': 0.566},
 '5 factor': {'BETAS': {"['Mkt-RF' 'SMB' 'HML' 'RMW' 'CMA']": [1.4684282047225827,
    0.9586200883174223,
    1.1725207410229357,
    -0.730198260942138,
    1.0976978992175697]},
  'R2': 0.5932}}

# Resultados

Lo que se hace con los modelos contra cada acción es una regresión lineal, para el modelo del CAPM es una regresión lineal simple donde solo se toma en cuenta el retorno del mercado menos la tasa libre de riesgo, para los otros dos modelos se utiliza una regresión lineal múltiple ya que se toman en cuenta otros factores que existen en el mercado, y para cada modelo se calcula el r2, que es una manera de saber que tan confiable y bueno es el modelo.

Para la mayoría de acciones el r2 va aumentando conforme aumentamos la cantidad de factores en el modelo. El cambio de r2 del CAPM contra el 3 factor suele ser bastante grande y significativo por lo que el modelo de 3 factor se puede considerar mejor y más completo al CAPM ya que su r2 suele ser bastante mayor indicando mayor confianza en el resultado.

El cambio entre el r2 del 3 factor contra el 5 factor suele ser muy pequeño, a pesar de aumentar la cantidad de factores en el modelo si bien lo confiable del modelo si es mayor, es un cambio muy pequeño para la cantidad de datos que se le agregan al modelo por lo que se considera que es excesivo utilizar el modelo 5 factor para la poca mejora.

Tomar en cuenta los factores de SMB, donde compara retornos de empresas chicas contra grandes y el HML que analiza los retornos de empresas de distinto book-to-value ratio; suele ser una buena idea ya que mejora el r2 lo que significa que estos factores son importantes en el mercado. Cuando se agregan el RMW donde ve empresas robustas contra débiles y el CMA donde analiza empresas conservadoras contra agresivas el incremento del r2 no es alto por lo que estos fatores no tienen mucho peso en el mercado, sin embargo, en ciertas acciones usar el modelo de 5 factor si mejora mucho el r2 por lo que puede ser útil hacerlo para ver en que tipo de empresas si tiene una mayor importancia.